In [ ]:
import pandas as pd
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog
from tqdm import tqdm
import time

def get_all_players_game_stats(season):


    # Get all players
    all_players = players.get_players()

    # DataFrame to store all player stats
    all_player_stats = pd.DataFrame()

    for player in tqdm(all_players):
        player_id = player['id']
        player_name = player['full_name']

        try:
            # Retrieve player game log for the specified season
            gamelog = playergamelog.PlayerGameLog(player_id=player_id, season=season)
            player_stats = gamelog.get_data_frames()[0]

            # Add player name, ID, and season for reference
            player_stats['Player Name'] = player_name
            player_stats['Player ID'] = player_id
            player_stats['Season'] = season

            # Append to the main DataFrame
            all_player_stats = pd.concat([all_player_stats, player_stats], ignore_index=True)

            # Sleep to avoid hitting the rate limit
            time.sleep(0.6)

        except Exception as e:
            print(f"Error fetching data for player {player_name} in season {season}: {e}")

    return all_player_stats

def generate_nba_seasons(start_year, end_year):
    seasons = []

    for year in range(start_year, end_year):
        # For each year, the NBA season is from year to year+1
        season = f"{year}-{str(year+1)[-2:]}"
        seasons.append(season)

    return seasons


# Define the seasons you are interested in
seasons = generate_nba_seasons(2022, 2023)

# Fetch all player game stats for the defined seasons
for season in seasons:
    all_player_game_stats = get_all_players_game_stats(season)
    all_player_game_stats_sorted = all_player_game_stats.sort_values(by='GAME_DATE') #sort by date
    all_player_game_stats.to_csv(f"all_player_game_stats_season_{season}", index=False)

print("Complete")

 58%|████████████████████████████████████████████▏                               | 2849/4900 [1:13:21<47:17,  1.38s/it]